In [6]:
!pip install google-api-python-client
!pip install hampel
!pip install pandas statsmodels
!pip install streamlit


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.7/240.7 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 30.3 MB/s eta 0:00:00
  Attempting uninstall

In [7]:
import os
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from hampel import hampel
from sklearn.metrics import r2_score
import altair as alt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import streamlit as st

import os
import json
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build

service_account_info = json.loads(os.environ['SERVICE_ACCOUNT'])
scopes = ['https://www.googleapis.com/auth/spreadsheets']
credentials = service_account.Credentials.from_service_account_info(service_account_info, scopes=scopes)
service = build('sheets', 'v4', credentials=credentials)
sheet = service.spreadsheets()

In [8]:
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1C9AKwU55jVA_ep3IiQHH6BpoAKQg--yNxcMQ2RLvVYc/edit#gid=1580399618'
spreadsheet_id = google_sheet_url.split('/')[5]


spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
sheets = [sheet['properties']['title'] for sheet in spreadsheet['sheets']]
print(sheets)


# get the columns you desire
range_name = 'UK Data (PURO 2024)'
result = sheet.values().get(spreadsheetId=spreadsheet_id,
                        range=range_name).execute()

values = result['values']

# contruct a Pandas DataFrame from the raw values that were returned from the Google Sheet
df = pd.DataFrame(values[0:], columns=values[0])
print(df)

['UK Data (PURO 2024)', 'Data Integrity (2024)', 'UK Data (2023)', 'Data Integrity (2023)', 'Missing LO contracts (PURO 2023)', 'Copy of England Data (2023)', 'Quarry Data', 'England Data (2023)', 'Data Entry Timeline', 'US Data (2023)', 'Rock Write-off (UK)', 'Pre-Newton Data', 'Agseq Data', 'CW Data', 'CW job status form', 'hiddenSheet']
          Job                          Site     Hauler   Spreader  Month End  \
0         Job                          Site     Hauler   Spreader  Month End   
1    J-001335          Blackford - Boreland        WHM   Paterson      01/24   
2    J-001316               Feithhill (III)       SARS       SARS      01/24   
3    J-001224       Nether Fordun - Dunning  Strathore  Patterson      01/24   
4   J-001393   Nether Fordun - Dunning (II)  Strathore  Patterson      02/24   
..        ...                           ...        ...        ...        ...   
57                           Newmilns (III)        WMH        FRM              
58   J-001195     

In [9]:
df_clean = df.loc[:,['Job','Site', 'Month End','Spreader Fuel Data','Loader Fuel Data','Task Data / GPS','Rock Spreads','Dropsite Photo (before)','Dropsite Photo (after)','Attestation','Landowner Contract']]
df_clean

Job                          Site  Month End  Spreader Fuel Data  \
0         Job                          Site  Month End  Spreader Fuel Data   
1    J-001335          Blackford - Boreland      01/24                Done   
2    J-001316               Feithhill (III)      01/24                Done   
3    J-001224       Nether Fordun - Dunning      01/24                Done   
4   J-001393   Nether Fordun - Dunning (II)      02/24                Done   
..        ...                           ...        ...                 ...   
57                           Newmilns (III)                    Not Started   
58   J-001195       Kingsbeck (Hangingshaw)                    Not Started   
59   J-001336          Blackford - Potholes                    Not Started   
60   J-001198        Kingsbeck (Waterworks)                    Not Started   
61                                Greenhill                    Not Started   

    Loader Fuel Data  Task Data / GPS  Rock Spreads  Dropsite Photo (before)  \
0   Loader Fuel Data  Task Data / GPS  Rock Spreads  Dropsite Photo (before)   
1            Missing             Done          Done  Not needed at this time   
2               Done             Done          Done  Not needed at this time   
3            Missing             Done          Done  Not needed at this time   
4               Done             Done          Done  Not needed at this time   
..               ...              ...           ...                      ...   
57       Not Started      Not Started   Not Started              Not Started   
58       Not Started      Not Started   Not Started              Not Started   
59       Not Started      Not Started   Not Started              Not Started   
60       Not Started      Not Started   Not Started              Not Started   
61       Not Started      Not Started   Not Started              Not Started   

    Dropsite Photo (after)              Attestation  Landowner Contract  
0   Dropsite Photo (after)              Attestation  Landowner Contract  
1                     Done  Not needed at this time                Done  
2                     Done  Not needed at this time                Done  
3                     Done  Not needed at this time                Done  
4                     Done  Not needed at this time                Done  
..                     ...                      ...                 ...  
57             Not Started              Not Started         Not Started  
58             Not Started              Not Started         Not Started  
59             Not Started              Not Started         Not Started  
60             Not Started              Not Started         Not Started  
61             Not Started              Not Started         Not Started  

[62 rows x 11 columns]

In [10]:
# counting the number of 'available', 'missing', 'partially missing', and 'issue' values in the dataframe
counts = df_clean.eq('Done').sum().rename('Available')
counts = counts.to_frame().assign(missing=df_clean.eq('Missing').sum().rename('Missing'),
                                   partially_missing=df_clean.eq('Partially Missing').sum().rename('Partially Missing'),
                                   issue=df_clean.eq('Issue').sum())

print(counts)

                         Available  missing  partially_missing  issue
Job                              0        0                  0      0
Site                             0        0                  0      0
Month End                        0        0                  0      0
Spreader Fuel Data              27        1                  1      2
Loader Fuel Data                27        4                  0      0
Task Data / GPS                 25        0                  0      0
Rock Spreads                    30        0                  0      1
Dropsite Photo (before)         19        2                  0      0
Dropsite Photo (after)          28        3                  0      0
Attestation                     21        0                  0      0
Landowner Contract              30        0                  0      0


In [30]:
# switching the rows and columns in the dataframe
counts_transposed = counts.T

# adding a row with the total number of missing, partially missing, and issue values
counts_transposed['Total'] = counts_transposed.sum(axis=1)
counts_transposed

# removing 'job', 'site', and 'month end' columns
counts_transposed = counts_transposed.drop(['Job', 'Site', 'Month End'], axis=1)
counts_transposed



Spreader Fuel Data  Loader Fuel Data  Task Data / GPS  \
Available                          27                27               25   
missing                             1                 4                0   
partially_missing                   1                 0                0   
issue                               2                 0                0   

                   Rock Spreads  Dropsite Photo (before)  \
Available                    30                       19   
missing                       0                        2   
partially_missing             0                        0   
issue                         1                        0   

                   Dropsite Photo (after)  Attestation  Landowner Contract  \
Available                              28           21                  30   
missing                                 3            0                   0   
partially_missing                       0            0                   0   
issue                                   0            0                   0   

                   Total  
Available            207  
missing               10  
partially_missing      1  
issue                  3

In [31]:
# Creating Streamlit Dashboard
st.set_page_config(
    page_title="UK Data Integrity (2024)",
    page_icon="🏂",
    layout="wide",
    initial_sidebar_state="expanded")

alt.themes.enable("dark")

ThemeRegistry.enable('dark')

In [37]:
counts_transposed_st = st.dataframe(counts_transposed)

# cols = st.columns([1, 1])

with cols[0]:
    fig = px.pie(counts_transposed, values= 'Total', names=('Available','Missing','Partially Missing','Issue'),
                 title='UK Data Integrity (2024)',
                 height=900, width=600)
    fig.update_layout(margin=dict(l=20, r=20, t=30, b=0),)
    st.plotly_chart(fig, use_container_width=True)

fig.show()

DeltaGenerator()


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=66942130-1354-4aab-90fa-c023179afef1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>